In [1]:
import pandas as pd
import numpy as np  
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score 

In [2]:
%load_ext rpy2.ipython

In [3]:
abbrev = pd.read_csv("data/abbrev.csv")
log_trans = abbrev.copy()

In [4]:
log_trans.columns.values

array(['Unnamed: 0', 'household_size', 'empl_agriculture',
       'empl_professional', 'empl_social', 'empl_services',
       'empl_manufacturing', 'empl_retail', 'prc_fam_poverty',
       'avg_income', 'prc_public_transp', 'population', 'pop_65_plus',
       'health_ins', 'county', 'state', 'area', 'domestic_passengers',
       'intl_passengers', 'deaths', 'ten_plus', 'order', 'density',
       'death_prc'], dtype=object)

In [5]:
log_trans = log_trans.drop(['Unnamed: 0'], axis=1)

In [6]:
# replace zeros with 0.0000001 to avoid -inf when apply log transformation
log_trans.loc[log_trans["population"] == 0, "population"] = 0.0000001
log_trans.loc[log_trans["density"] == 0, "density"] = 0.0000001
log_trans.loc[log_trans["deaths"] == 0, "deaths"] = 0.0000001
log_trans.loc[log_trans["death_prc"] == 0, "death_prc"] = 0.0000001

In [7]:
log_trans[["population", "density", "deaths", "death_prc"]] = log_trans[["population", "density", "deaths", "death_prc"]].transform(np.log, axis=0)

In [8]:
log_trans[["population", "density", "deaths", "death_prc"]].head()

,population,density,deaths,death_prc
0,11.687626,5.325737,-16.118096,-16.118096
1,11.927456,4.690814,0.000000,-11.927456
2,12.984466,4.888684,1.098612,-11.885854
3,11.509881,4.645273,0.000000,-11.509881
4,12.100934,3.864673,1.098612,-11.002322


In [9]:
log_trans[["population", "density", "deaths", "death_prc"]].describe()

,population,density,deaths,death_prc
count,823.000000,823.000000,823.000000,823.000000
mean,12.184464,5.674172,-2.059859,-11.674516
std,0.900335,1.204975,7.643905,2.583123
min,11.044632,1.475446,-16.118096,-16.118096
25%,11.471227,4.943125,0.000000,-12.253978
50%,11.978777,5.603942,1.098612,-10.963498
75%,12.708739,6.324017,2.564949,-9.968105
max,16.128592,10.231016,9.348100,-6.595493


In [10]:
predictors = ['household_size', 'empl_agriculture', 'empl_professional','empl_social', 'empl_services', 'empl_manufacturing', 'empl_retail',
              'prc_fam_poverty', 'avg_income', 'prc_public_transp', 'population', 'pop_65_plus', 'health_ins', 'area', 
              'domestic_passengers', 'intl_passengers', 'ten_plus', 'order', 'density']

In [11]:
# standard score normalize the predictor data
scaler = StandardScaler().fit(abbrev[predictors])
X = pd.DataFrame(scaler.transform(abbrev[predictors]), columns=predictors)
X_log = pd.DataFrame(scaler.transform(log_trans[predictors]), columns=predictors)

In [12]:
X.describe()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,prc_fam_poverty,avg_income,prc_public_transp,population,pop_65_plus,health_ins,area,domestic_passengers,intl_passengers,ten_plus,order,density
count,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02,8.230000e+02
mean,-1.563148e-17,9.618336e-17,-2.695292e-16,9.204194e-16,-1.220166e-16,1.160136e-17,2.441681e-16,-9.712765e-17,-3.283454e-16,-1.510875e-17,2.590071e-17,1.413072e-16,-9.517363e-15,-2.158392e-17,4.532624e-17,-2.784326e-16,4.262825e-17,1.342250e-16,-4.316785e-18
std,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00,1.000608e+00
min,-2.763242e+00,-7.138190e-01,-2.817129e+00,-5.030197e+00,-3.468572e+00,-1.894605e+00,-5.734296e+00,-1.826346e+00,-2.246612e+00,-4.516647e-01,-4.278334e-01,-2.109147e+00,-9.930490e+00,-5.813653e-01,-4.529022e-01,-2.621418e-01,-2.632091e+00,-2.902770e+00,-3.973592e-01
25%,-7.217483e-01,-5.129159e-01,-6.764601e-01,-6.319020e-01,-6.104182e-01,-7.365944e-01,-6.214260e-01,-7.363487e-01,-6.654138e-01,-3.811994e-01,-3.761171e-01,-6.368384e-01,-4.111277e-02,-3.481383e-01,-4.529022e-01,-2.621418e-01,-5.959393e-01,-5.518204e-01,-3.175188e-01
50%,-1.613383e-01,-3.283267e-01,-1.428487e-01,-1.449382e-01,-1.322056e-01,-1.707755e-01,-4.889213e-02,-1.439586e-01,-1.787571e-01,-2.872456e-01,-2.776482e-01,-9.348539e-02,3.293886e-01,-2.423046e-01,-4.192704e-01,-2.621418e-01,4.035811e-02,-1.369469e-01,-2.403517e-01
75%,4.791302e-01,1.355948e-01,5.145289e-01,5.198946e-01,4.287507e-01,5.613396e-01,5.505121e-01,5.550616e-01,4.794669e-01,-2.887279e-02,-1.170245e-02,4.479043e-01,4.880274e-01,-8.815729e-02,-2.068709e-01,-2.621418e-01,6.766555e-01,8.310913e-01,-7.206297e-02
max,6.083230e+00,8.759080e+00,5.592866e+00,4.703169e+00,8.005615e+00,5.621149e+00,4.852515e+00,4.832118e+00,5.120250e+00,1.267038e+01,1.516502e+01,9.451954e+00,7.808444e-01,1.294436e+01,4.705967e+00,9.753145e+00,3.094586e+00,1.660838e+00,1.591125e+01


In [13]:
y = abbrev["deaths"]
y_prc = abbrev["death_prc"]
y_log = log_trans["deaths"]
y_log_prc = log_trans["death_prc"]

### Linear Regression

In [14]:
lrm = LinearRegression().fit(X, y)
lrm.score(X, y)

0.5403920042426926

In [15]:
for a,b in zip(predictors, lrm.coef_):
    print(a, "\t", b)

household_size 	 28.968067057984335
empl_agriculture 	 6.918465800092834
empl_professional 	 -71.60569497930568
empl_social 	 19.076759054195847
empl_services 	 1.7843686519807638
empl_manufacturing 	 -5.625714042373956
empl_retail 	 1.6856880396875265
prc_fam_poverty 	 -51.1263705754051
avg_income 	 -28.507791118862666
prc_public_transp 	 -19.995121965294818
population 	 146.7841311219609
pop_65_plus 	 29.416274508758093
health_ins 	 -0.08832137685987934
area 	 -14.830700261506749
domestic_passengers 	 -111.63457397445528
intl_passengers 	 115.32284826250077
ten_plus 	 -25.881627433347003
order 	 -5.716752898122561
density 	 238.519268458166


In [16]:
lrm_prc = LinearRegression().fit(X, y_prc)
lrm_prc.score(X, y_prc)

0.4176098844380915

In [17]:
lrm_logy = LinearRegression().fit(X, y_log)
lrm_logy.score(X, y_log)

0.36644400597395915

In [18]:
lrm_log = LinearRegression().fit(X_log, y_log)
lrm_log.score(X_log, y_log)

0.3853187097087529

In [19]:
lrm_log_prc = LinearRegression().fit(X_log, y_log_prc)
lrm_log_prc.score(X_log, y_log_prc)

0.3735355664309719

In [20]:
lrm_log = LinearRegression().fit(X_log, y)
lrm_log.score(X_log, y)

0.37270174570160786

In [21]:
lrm_log = LinearRegression().fit(X_log, y_prc)
lrm_log.score(X_log, y_prc)

0.4207429978378541

In [22]:
lrm_raw = LinearRegression().fit(abbrev[predictors], y)
lrm_raw.score(abbrev[predictors], y)

0.5403920042426926

In [23]:
lrm_raw = LinearRegression().fit(abbrev[predictors], y_prc)
lrm_raw.score(abbrev[predictors], y_prc)

0.4176098844380915

In [24]:
lrm_raw = LinearRegression().fit(abbrev[predictors], y_log)
lrm_raw.score(abbrev[predictors], y_log)

0.36644400597395915

In [25]:
lrm_raw = LinearRegression().fit(abbrev[predictors], y_log_prc)
lrm_raw.score(abbrev[predictors], y_log_prc)

0.36566570880352633

### Elastic Net

#### ElasticNetCV

In [26]:
encv = ElasticNetCV(normalize=True, max_iter=60000, random_state=101, cv=5)
encv.fit(X, y)

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=60000, n_alphas=100, n_jobs=None,
             normalize=True, positive=False, precompute='auto',
             random_state=101, selection='cyclic', tol=0.0001, verbose=0)

In [27]:
encv.score(X, y)
# yikes, bad

0.19587609278319784

In [28]:
# try with cv=3
encv = ElasticNetCV(normalize=True, max_iter=60000, random_state=101, cv=3)
encv.fit(X, y)
encv.score(X, y)
# and not better

0.15756278222573383

In [29]:
# try with cv=7
encv = ElasticNetCV(normalize=True, max_iter=60000, random_state=101, cv=7)
encv.fit(X, y)
encv.score(X, y)
# and not better

0.19587609278319784

#### ElasticNet with GridSearchCV

In [41]:
param_grid = [{'l1_ratio':[0.1, 0.3, 0.5, 0.7, 0.9]}]
enm = ElasticNet(normalize=True, max_iter=60000, random_state=101)
scorer = make_scorer(r2_score)
search = GridSearchCV(enm, param_grid, cv=3, scoring=scorer).fit(abbrev[predictors], abbrev["deaths"])

In [42]:
search

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=60000, normalize=True,
                                  positive=False, precompute=False,
                                  random_state=101, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(r2_score), verbose=0)

In [43]:
search.best_params_

{'l1_ratio': 0.9}

In [44]:
search.best_estimator_

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=60000, normalize=True, positive=False, precompute=False,
           random_state=101, selection='cyclic', tol=0.0001, warm_start=False)

In [45]:
search.best_score_
# yikes, not good, ElasticNetCV
# with CV=10 l1_ratio=0.9, score=0.0373
# with CV=5 l1_ratio=0.9, score=0.0544
# with CV=3 l1_ratio=0.9, score=0.0793

0.07931775070155091

In [ ]:
enm = ElasticNet(random_state=101, max_iter=60000)
enm.fit(X, y)
enm.score(X, y)
# cross validation hard, likes overfitting :)

In [ ]:
for a, b in zip(predictors, enm.coef_):
    print(a, "\t", b)

### Random Forest

In [ ]:
# need to rerun with a pipeline to apply normalization to subset in the CV fold used for training!

In [ ]:
rfm = RandomForestRegressor(random_state=1001, oob_score=True)
param_grid = {
    'max_depth':[5, 10, 15, 20, 25, 30],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [50, 100, 200, 300, 1000]
}
grid_search = GridSearchCV(estimator=rfm, param_grid=param_grid, cv=3, scoring=scorer)
grid_search.fit(X, y)


In [ ]:
grid_search.best_params_

In [ ]:
pred = grid_search.best_estimator_.predict(X)
r2_score(y, pred)

In [ ]:
exam[["pred", "deaths"]]

In [ ]:
exam["delta"] = exam["deaths"] - exam["pred"]

In [ ]:
exam[exam["delta"] > 5].shape[0]

In [ ]:
exam[exam["delta"] < -5].shape[0]

In [ ]:
exam[(exam["delta"] > -5) & (exam["delta"] < 5)].shape[0] # more are within 5 deaths of actual count than aren't

In [ ]:
exam[exam["delta"] > 10].shape[0]

In [ ]:
exam[exam["delta"] < -10].shape[0]

In [ ]:
exam[(exam["delta"] > -10) & (exam["delta"] < 10)].shape[0] # more than twice as many are within 10 deaths of actual count than those that aren't

In [ ]:
exam = X
exam["pred"] = pred
exam["deaths"] = y

In [ ]:
exam.head()

In [ ]:
#rfm.fit(X, y)
#rfm.score(X, y)
# .665


In [ ]:
for a, b in zip(predictors, grid_search.best_estimator_.feature_importances_):
    print(a, "\t", b)

In [ ]:
estimator = grid_search.best_estimator_.estimators_[5]
# https://medium.com/@anthonycarlleston/visualizing-the-decisiontrees-in-randomforestregressor-in-a-pipeline-with-python-f5519f80e3f8
# https://scikit-learn.org/stable/modules/tree.html

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import os

os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"
dot_data = StringIO()
export_graphviz(estimator, feature_names=predictors, out_file=dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
Image(graph.create_png())